In [1]:
import time
import typing as t

from openpyxl import load_workbook
from datetime import datetime
from sqlalchemy.sql import func
from sqlalchemy.orm import Session

from app.core.config import settings
from app.db.session import SessionLocal

from app import crud, models, schemas

db = SessionLocal()

/usr/local/lib/python3.7/site-packages/jose/backends/cryptography_backend.py:18: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes, int_to_bytes


In [2]:
def get_file(file="reklaliste.xlsx") -> load_workbook:
    wb_file = load_workbook(str(settings.SERVER_BASE_DIR) + '/' + file)
    i = 0
    for sheet in wb_file.sheetnames:
        i += 1
        if sheet.lower() == "liste wertreklas":
            break
        wb_file.active = i
    return wb_file.active

In [14]:
def fetch_claim_data(row: t.List) -> t.List:
    data = []
    for cell in row:
        data.append(cell)
    return data

In [13]:
def create_claim(*, data: t.List) -> t.Union[schemas.claim.ClaimBase, None]:
    claim_in = schemas.claim.ClaimBase()

    if isinstance(data[1], float):
        contract_nr = str(data[1]).split(".0")[0]
        claim_in.contract_nr = int(contract_nr)
    if isinstance(data[2], float) or isinstance(data[2], int):
        claim_in.discharge = float(data[2])
    if isinstance(data[3], float) or isinstance(data[3], int):
        claim_in.bill = float(data[3])
    if isinstance(data[7], datetime):
        claim_in.created_at = datetime.date(data[7])
    if isinstance(data[11], str):
        claim_in.kind = data[11].upper()
        try:
            claim_in.kind = models.ClaimKind(claim_in.kind).name
        except ValueError:
            claim_in.kind = None

    if claim_in.contract_nr is not None and claim_in.bill is not None and claim_in.created_at is not None:
        return claim_in
    return None

In [15]:
def iter_claim_rows(file: load_workbook) -> t.List[schemas.Claim]:
    i = 0
    for row in file.iter_rows(values_only=True, max_col=15):
        i += 1
        if i  == 10:
            break
        data = fetch_claim_data(row)
        echo = create_claim(data=data)
        if echo is not None:
            yield echo

In [16]:
rows = iter_claim_rows(file=get_file())

/usr/local/lib/python3.7/site-packages/openpyxl/styles/stylesheet.py:214: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [17]:
for row in rows:
    print(row)

contract_nr=1023309 bill=69.97 discharge=None kind='SUPPLIER' ticket_id=None owner_id=None store_internal_id=None created_at=datetime.date(2020, 1, 2)
contract_nr=1068349 bill=97.62 discharge=-120.8 kind='ASSEMBLER' ticket_id=None owner_id=None store_internal_id=None created_at=datetime.date(2020, 1, 2)
contract_nr=1099803 bill=93.78 discharge=-1142.74 kind='ASSEMBLER' ticket_id=None owner_id=None store_internal_id=None created_at=datetime.date(2020, 1, 2)
contract_nr=1174687 bill=342.36 discharge=None kind='SELLER' ticket_id=None owner_id=None store_internal_id=None created_at=datetime.date(2020, 1, 2)
contract_nr=1174687 bill=38.03 discharge=None kind='SELLER' ticket_id=None owner_id=None store_internal_id=None created_at=datetime.date(2020, 1, 2)
contract_nr=1264886 bill=9.17 discharge=None kind='CUSTOMER' ticket_id=None owner_id=None store_internal_id=None created_at=datetime.date(2020, 1, 2)
contract_nr=1302405 bill=44.07 discharge=None kind='SUPPLIER' ticket_id=None owner_id=None